In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from CA1_Yolo import *

Using TensorFlow backend.


In [2]:
img_dir = "../data/image/"
anno_dir = "../data/annotation/"

In [3]:
all_anno, labels = read_annotation_files(img_dir, anno_dir)

In [8]:
anchor_boxes = generateAnchorBoxes(all_anno, labels)

iter: 1 distances: 1724.718957580812
iter: 2 distances: 122.29815449055585
iter: 3 distances: 36.11777361952636
iter: 4 distances: 25.41096478200987
iter: 5 distances: 28.237762116892284
iter: 6 distances: 25.397598191444274
iter: 7 distances: 20.678023443341402
iter: 8 distances: 14.208581515533968
iter: 9 distances: 13.684299234718033
iter: 10 distances: 11.162773952417442
iter: 11 distances: 10.757528896403585
iter: 12 distances: 6.493985519942916
iter: 13 distances: 4.4822730061492955
iter: 14 distances: 5.415841159088774
iter: 15 distances: 10.829251410778834
iter: 16 distances: 4.352434647088011
iter: 17 distances: 5.666277264505064
iter: 18 distances: 6.103974379170538
iter: 19 distances: 11.906342684702008
iter: 20 distances: 7.167363804738278
iter: 21 distances: 8.179619400914973
iter: 22 distances: 7.737550992468146
iter: 23 distances: 1.602589989784664
iter: 24 distances: 1.5880383449745874
iter: 25 distances: 4.952313741171151
iter: 26 distances: 4.739761977435743
iter: 27 

In [9]:
anchor_boxes

[(32, 25),
 (40, 62),
 (52, 35),
 (56, 104),
 (70, 50),
 (89, 73),
 (90, 124),
 (114, 36),
 (114, 92)]

iter: 1 distances: 1705.940767736839
iter: 2 distances: 139.66237677303428
iter: 3 distances: 67.95478957946011
iter: 4 distances: 45.75059930334519
iter: 5 distances: 39.092353490628376
iter: 6 distances: 25.366935682753194
iter: 7 distances: 33.908221669722266
iter: 8 distances: 25.255558609073613
iter: 9 distances: 23.188574057487735
iter: 10 distances: 25.291832609227363
iter: 11 distances: 23.038867265823107
iter: 12 distances: 36.505286964403936
iter: 13 distances: 27.47180293190275
iter: 14 distances: 13.974780817640042
iter: 15 distances: 8.272912199959963
iter: 16 distances: 4.041807917845413
iter: 17 distances: 5.430091172205444
iter: 18 distances: 1.3793129412886902
iter: 19 distances: 3.0560908313895823
